<a href="https://colab.research.google.com/github/bryanpurba/Coursera_Capstone/blob/master/W2_Peer_Graded_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is used for Applied Data Science Capstone peer-graded assignment 

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Import Necessary Packages

In [3]:
import urllib
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Using BeautifulSoup to Scraping HTML Tag

In [4]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
# print(soup.prettify())

# Print all tables in HTML Tag to find out our target Table

In [5]:
all_tables=soup.find_all("table")
# all_tables

# From previous cell we findout our target table has class wikitalbe sortable, use that attribute to acquire target table

In [6]:
right_table=soup.find('table', class_='wikitable sortable')
# right_table

# Because we don't need table header, we can skip tag 'th' and find tag 'td' only, from wikipedia pages and previous cell we findout that each rows has 3 children that is [ Postal Code, Borough, Neighborhood ], with this format we can assign values from td tag to each categories

In [7]:
postal_code = []
borough = []
neighborhood = []

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postal_code.append(cells[0].find(text=True).rstrip('\n'))
        borough.append(cells[1].find(text=True).rstrip('\n'))
        neighborhood.append(cells[2].find(text=True).rstrip('\n'))

# Convert the acquired datas to pandas DataFrame

In [8]:
df = pd.DataFrame()
df['PostalCode'] = postal_code
df['Borough'] = borough
df['Neighborhood'] = neighborhood
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


# Drop all "Not assigned" values in Borough Column using df.drop

In [9]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Combine rows that has same postal code but have different Neighborhood using groupby and apply methods. However, i findout there is no rows that have different Neighborhood and same Postal Code

In [10]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.columns = ['PostalCode','Borough','Neighborhood']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# If Borough columns has value and has "Not assigned" value in Neighborhood columns then replace "Not assigned value" to value in Borough columns

In [11]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
df.shape

(103, 3)

# Finished convert table from scraped linked wikipedia page to pandas dataframe. However, we have different visual of df.head() from assignment example, it is because i use groupby methods that change index order